In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
from report_utils import excel_report

import numpy as np
from log import log

In [2]:
sql = '''
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case 
        	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
        	else 'ПРОЧИЕ' 
       end "name",
	   case
	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 0610000 and 0629999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3050000 and 3079999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3130000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3180000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3449999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3870000 and 3889999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6600000 and 6699999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7120000 and 7179999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7600000 and 7699999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0200000 and 0269999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0380000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3700000 and 3799999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 1100000 and 1249999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 3200000 and 3339999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3300000 and 3399999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3439999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3460000 and 3499999)
	   			 or (substring(per.phone, 2, 3) = '909' and cast(substring(per.phone, 5, 7) as numeric) between 3060000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 2200000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 3900000 and 3999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 8500000 and 9399999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6200000 and 6499999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6800000 and 6999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0400000 and 0499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 2400000 and 2499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4000000 and 4499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4500000 and 4999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '937' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3299999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 6650000 and 6689999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 0600000 and 0699999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 8900000 and 8999999)
	   			 or (substring(per.phone, 2, 3) = '960' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0899999)
	   			 or (substring(per.phone, 2, 3) = '962' and cast(substring(per.phone, 5, 7) as numeric) between 5480000 and 5799999)
	   			 or (substring(per.phone, 2, 3) = '963' and cast(substring(per.phone, 5, 7) as numeric) between 1200000 and 1259999)
	   			 or (substring(per.phone, 2, 3) = '987' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			) and length(phone) = 11
	   			)
	   		then 'Республика Татарстан'
	   		else 'Другой регион'
	   end region,
	   DATE(a.finished + 3 * interval '1 hour')
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join promotion promo on a.promotion_id = promo.id
where a.finished at time zone 'UTC' > '2020-03-30'
and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second')
and a.good
	and not (
			a.patient_id in (
							select patient_id from patient_categories pcat
							inner join reference rf 
							on rf.id = pcat.reference_id
							where rf.code = 'TEST'
			)
			or lower(a.report_comment) = 'тест'
	)


'''

In [3]:
connection = UniversalConnection('../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-04-08 17:12:25 connect to postgres database using config file "../../.credentials/telemed/prom.cfg"
2020-04-08 17:12:25 creating ssh tunnel to 172.16.100.19 as root...
2020-04-08 17:12:35 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 39335
2020-04-08 17:12:35 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [4]:
data = connection.query(sql)

2020-04-08 17:12:35 @telemed query:
                    select a.id appoint_id,
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case 
				            	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
				            	else 'ПРОЧИЕ' 
				           end "name",
				    	   case
				    	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
				    	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) bet

In [5]:
connection.close()

2020-04-08 17:12:36 @telemed: closing postgres connection...
2020-04-08 17:12:36 closing ssh tunnel to 172.16.100.19...


In [6]:
df = pd.DataFrame(data)

In [7]:
df.head()

,appoint_id,person_id,phone,name,region,date
0,127919,2546091,79518565014,VEBMED,Другой регион,2020-03-31
1,136013,3201622,79089255484,ПРОЧИЕ,Другой регион,2020-04-07
2,128137,1141636,79180989263,ПРОЧИЕ,Другой регион,2020-03-31
3,135952,3201364,79911116364,VEBMED,Другой регион,2020-04-07
4,127960,2626621,79149123969,ПРОЧИЕ,Другой регион,2020-03-31


In [8]:
df.groupby(['date','region', 'name'])['person_id'].count().to_frame().reset_index() #.sort_values(by=['date', 'region', 'name'], ascending=True)

,date,region,name,person_id
0,2020-03-30,Другой регион,VEBMED,34
1,2020-03-30,Другой регион,ПРОЧИЕ,489
2,2020-03-30,Республика Татарстан,VEBMED,1
3,2020-03-30,Республика Татарстан,ПРОЧИЕ,7
4,2020-03-31,Другой регион,VEBMED,107
5,2020-03-31,Другой регион,ПРОЧИЕ,452
6,2020-03-31,Республика Татарстан,VEBMED,2
7,2020-03-31,Республика Татарстан,ПРОЧИЕ,12
8,2020-04-01,Другой регион,VEBMED,187
9,2020-04-01,Другой регион,ПРОЧИЕ,506


In [9]:
df2 = pd.DataFrame(columns = ['date', 'total', 'web', 'tatar', 'web-tatar'])

In [10]:
df2

,date,total,web,tatar,web-tatar


In [11]:
d = dict()

In [12]:
for i in range(df.shape[0]):
    #print(i)
    if df['name'][i] == 'VEBMED':
        d[df['date'][i]] = d.get(df['date'][i], [0, 0, 0, 0, 0, 0]) 
        d[df['date'][i]][0] += 1
        d[df['date'][i]][1] += 1
        
        if df['region'][i] == 'Республика Татарстан':
            d[df['date'][i]][4] += 1
            d[df['date'][i]][3] += 1
            
    if df['name'][i] != 'VEBMED':
        d[df['date'][i]] = d.get(df['date'][i], [0, 0, 0, 0, 0, 0]) 
        d[df['date'][i]][0] += 1
        d[df['date'][i]][2] += 1
        
        if df['region'][i] == 'Республика Татарстан':
            d[df['date'][i]][3] += 1
            d[df['date'][i]][5] += 1

In [13]:
d

{datetime.date(2020, 3, 31): [573, 109, 464, 14, 2, 12],
 datetime.date(2020, 4, 7): [954, 503, 451, 29, 17, 12],
 datetime.date(2020, 4, 4): [747, 331, 416, 13, 8, 5],
 datetime.date(2020, 4, 5): [737, 358, 379, 42, 37, 5],
 datetime.date(2020, 4, 6): [1004, 549, 455, 44, 30, 14],
 datetime.date(2020, 3, 30): [531, 35, 496, 8, 1, 7],
 datetime.date(2020, 4, 2): [738, 195, 543, 10, 5, 5],
 datetime.date(2020, 4, 1): [703, 192, 511, 10, 5, 5],
 datetime.date(2020, 4, 3): [843, 270, 573, 17, 5, 12]}

In [14]:
report = pd.DataFrame.from_dict(d, orient='index').sort_index()

In [15]:
report.columns = ['Всего консультаций', 'Из них ВЭБ','Прочие', 'Всего Татарстан', 'Из них ВЭБ-Татарстан', 'Прочие']

In [16]:
report

,Всего консультаций,Из них ВЭБ,Прочие,Всего Татарстан,Из них ВЭБ-Татарстан,Прочие
2020-03-30,531,35,496,8,1,7
2020-03-31,573,109,464,14,2,12
2020-04-01,703,192,511,10,5,5
2020-04-02,738,195,543,10,5,5
2020-04-03,843,270,573,17,5,12
2020-04-04,747,331,416,13,8,5
2020-04-05,737,358,379,42,37,5
2020-04-06,1004,549,455,44,30,14
2020-04-07,954,503,451,29,17,12


In [17]:
report.to_csv('Tatarstan_daily.csv')